In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# DataFrame analyses
from src.DataFrameAnalyses import DataFramePlotGenerator

# Simulation
from src.simulation.ProductionSimulation import ProductionSimulation

pd.set_option('display.min_rows', 21)
pd.set_option('display.max_rows', 25)


In [ ]:
from decimal import Decimal
from src.domain.Collection import LiveJobCollection
from src.domain.Query import JobQuery
from src.DataFrameEnrichment import DataFrameEnrichment

In [ ]:
source_name= "Fisher and Thompson 10x10"
max_bottleneck_utilization=Decimal("0.85")

# I. Due Dates

In [ ]:
jobs = JobQuery.get_by_source_name_and_max_bottleneck_utilization(
    source_name=source_name,
    max_bottleneck_utilization=max_bottleneck_utilization
)

# Collection with jobs
jobs_collection = LiveJobCollection(jobs)
jobs_collection.to_jobs_dataframe()

## A) Simulation

In [ ]:
for job in jobs_collection.values():
    for operation in job.operations:
        operation.sim_duration = operation.duration

In [ ]:
simulation = ProductionSimulation(verbose = False, with_earliest_start=True)
simulation.run(jobs_collection, start_time = 0, end_time = None)

In [ ]:
finished_operations = simulation.get_finished_operation_collection()
df_fcfs = finished_operations.to_operations_dataframe()
df_fcfs


In [ ]:
df_jobs_times_temp = finished_operations.to_jobs_metrics_dataframe()
df_jobs_times_temp


In [ ]:
fig = DataFramePlotGenerator.get_elapsed_time_density_plot_figure_side_by_side(
    df_jobs_times_temp,
    routing_filter=["01-01", "01-05"],
    x_max=5000
)
output_path = "elapsed_time_density.pdf"
fig.savefig(output_path, format="pdf", bbox_inches="tight")
plt.show()




## B) Generation of deadlines using log-normal distribution

In [ ]:
df_jobs_times = DataFrameEnrichment.add_groupwise_lognormal_due_dates_by_group_mean(df_jobs_times_temp, sigma=0.25)
df_jobs_times

####  Plot of the new deadlines

In [ ]:
fig = DataFramePlotGenerator.get_scheduling_window_density_plot_figure_side_by_side(
    df_jobs_times,
    routing_filter=["01-01", "01-05"], x_max=5000
)

output_path = "scheduling_window_density.pdf"
fig.savefig(output_path, format="pdf", bbox_inches="tight")

plt.show()

#### Ensuring that the processing time is achievable to a certain extent in the best-case scenario

In [ ]:
# min. 100 % of processing time for each job is covered in best-case
df_jobs_times_final = DataFrameEnrichment.ensure_reasonable_due_dates(df_jobs_times, min_coverage = 1.0)

diff_count = (df_jobs_times_final["Due Date"] > df_jobs_times["Due Date"]).sum()

# Info: Compare the "Due Date" column
print(f"Number of different entries in 'Due Date': {diff_count}")

df_jobs_times_final

# II. Transition Times

In [ ]:
df_fcfs

In [ ]:
# noinspection PyProtectedMember
df_transition_times = DataFrameEnrichment._compute_transition_times_per_job_operation_backward(df_fcfs)
df_transition_times

In [ ]:
df_avg_transition_times = DataFrameEnrichment.compute_avg_transition_times_per_machine_backward(df_fcfs)
df_avg_transition_times

In [ ]:
transitions_df = finished_operations.to_transition_time_dataframe()
transitions_df

In [ ]:
waiting_df = finished_operations.to_waiting_time_dataframe()
waiting_df

In [ ]:
DataFrameEnrichment.aggregate_mean_per_group(
    transitions_df,
    group_column = "Machine",
    value_column = "Transition Time",
    new_column_name = "Ø Transition Time",
)

In [ ]:
DataFrameEnrichment.aggregate_mean_per_group(
    waiting_df,
    group_column = "Machine",
    value_column = "Waiting Time",
    new_column_name = "Ø Waiting Time",
)